In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

Loading Dataset

In [2]:
df = pd.read_csv("dataset.csv")

In [4]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
df = df.drop(["RowNumber","CustomerId","Surname"],axis=1)

Changing Categorical to Numerical

In [11]:
label_encoder_gender = LabelEncoder()
df["Gender"] = label_encoder_gender.fit_transform(df["Gender"])

In [15]:
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder_geography = OneHotEncoder()
geo_encoded=one_hot_encoder_geography.fit_transform(df[["Geography"]])

In [16]:
one_hot_encoder_geography.get_feature_names_out(["Geography"])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [19]:
geo_encoded.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [22]:
geo_encoded_df = pd.DataFrame(geo_encoded.toarray(),columns=one_hot_encoder_geography.get_feature_names_out(["Geography"]))

In [23]:
## Combining the encoded columns with the original data
df = pd.concat([df.drop("Geography",axis=1),geo_encoded_df],axis=1)

In [24]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [ ]:
## Saving the Scaler Models
with open("scaler_models/label_encoder_gender.pkl","wb") as file:
    pickle.dump(label_encoder_gender,file)

with open("scaler_models/one_hot_encoder_geography.pkl","wb") as file:
    pickle.dump(one_hot_encoder_geography,file)

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CreditScore        10000 non-null  int64  
 1   Gender             10000 non-null  int32  
 2   Age                10000 non-null  int64  
 3   Tenure             10000 non-null  int64  
 4   Balance            10000 non-null  float64
 5   NumOfProducts      10000 non-null  int64  
 6   HasCrCard          10000 non-null  int64  
 7   IsActiveMember     10000 non-null  int64  
 8   EstimatedSalary    10000 non-null  float64
 9   Exited             10000 non-null  int64  
 10  Geography_France   10000 non-null  float64
 11  Geography_Germany  10000 non-null  float64
 12  Geography_Spain    10000 non-null  float64
dtypes: float64(5), int32(1), int64(7)
memory usage: 976.7 KB


In [ ]:
#df.to_csv("dataset.csv",index=False)

Dividing the Dataset into Dependent and Independent Features

In [36]:
X = df.drop("Exited",axis=1)
y = df["Exited"]

Splitting the dataset into traintestsplit


In [38]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [39]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [41]:
with open("std_scaler.pkl","wb") as file:
    pickle.dump(scaler,file)

## ANN Implementation

In [43]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [45]:
X_train.shape[1]

12

In [50]:
## Building our ANN Model
model = Sequential([
    Dense(64,activation="relu",input_shape=(X_train.shape[1],)),## HL1
    Dense(32,activation="relu"),## HL2
    Dense(1,activation="sigmoid")## output layer
])

In [51]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 64)                832       
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [53]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
loss = tf.keras.losses.BinaryCrossentropy()

In [54]:
## Compiling the model
model.compile(optimizer=opt,loss=loss,metrics=["accuracy"])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
## Setting up the Tensorboard
log_dir = "logs/fit/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [60]:
## Setting up Early Stopping
early_stopping_callback = EarlyStopping(monitor="val_loss",patience=10,restore_best_weights=True)

In [61]:
## Training the model
history = model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=100,callbacks=[tensorflow_callback,early_stopping_callback])

Epoch 1/100
219/219 [==============================] - 0s 2ms/step - loss: 0.5064 - accuracy: 0.7924 - val_loss: 0.4916 - val_accuracy: 0.8053
Epoch 2/100
219/219 [==============================] - 0s 2ms/step - loss: 0.5076 - accuracy: 0.7924 - val_loss: 0.4984 - val_accuracy: 0.8040
Epoch 3/100
219/219 [==============================] - 0s 2ms/step - loss: 0.5112 - accuracy: 0.7924 - val_loss: 0.4943 - val_accuracy: 0.8053
Epoch 4/100
219/219 [==============================] - 0s 2ms/step - loss: 0.5121 - accuracy: 0.7920 - val_loss: 0.4941 - val_accuracy: 0.8053
Epoch 5/100
219/219 [==============================] - 0s 2ms/step - loss: 0.5112 - accuracy: 0.7924 - val_loss: 0.4909 - val_accuracy: 0.8053
Epoch 6/100
219/219 [==============================] - 0s 2ms/step - loss: 0.5106 - accuracy: 0.7924 - val_loss: 0.4928 - val_accuracy: 0.8053
Epoch 7/100
219/219 [==============================] - 0s 2ms/step - loss: 0.5107 - accuracy: 0.7924 - val_loss: 0.4938 - val_accuracy: 0.8053

In [62]:
model.save("models/model.h5")

c:\Users\HP\OneDrive\Documents\CustomerChurnPrediction\churnpred\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [66]:
## Load tensorboard extension
%reload_ext tensorboard

In [67]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 10660), started 0:02:32 ago. (Use '!kill 10660' to kill it.)